In [3]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
def dx_dt(t, x, y, z, sigma, b, r):
    return sigma * (y - x)

def dy_dt(t, x, y, z, sigma, b, r):
    return x * (r - z) - y

def dz_dt(t, x, y, z, sigma, b, r):
    return x * y - b * z

In [6]:
sigma = 10
b = 8/3
rs = [0.5, 10, 24.06, 30]
dt = 0.001
max_t = 20

In [7]:
def draw(solver):
    for r in rs:
        ts, xs, ys, zs = solver(r)

        print('r = {0}'.format(r))
        plt.plot(ts, xs, 'red')
        plt.show()
        plt.plot(ts, ys, 'green')
        plt.show()
        plt.plot(ts, zs, 'blue')
        plt.show()

        fig = go.Figure(
            data=[
                go.Scatter3d(
                    x=xs,
                    y=ys,
                    z=zs,
                    marker=go.Marker(
                        size=1
                    )
                )
            ]
        )
        fig.show()

In [8]:
def RK4(r, t0=0, x0=13.41265629, y0=13.46430003, z0=33.46156416):
    t = t0
    x = x0
    y = y0
    z = z0
    
    ts = [t0]
    xs = [x0]
    ys = [y0]
    zs = [z0]
    
    while t < max_t:       
        k1 = dx_dt(t           , x                , y, z, sigma, b, r)
        k2 = dx_dt(t + 0.5 * dt, x + 0.5 * dt * k1, y, z, sigma, b, r)
        k3 = dx_dt(t + 0.5 * dt, x + 0.5 * dt * k2, y, z, sigma, b, r)
        k4 = dx_dt(t + dt      , x + dt * k3      , y, z, sigma, b, r)
        x_i = dt * (k1 + 2 * k2 + 2 * k3 + k4) / 6 
        
        k1 = dy_dt(t           , x, y                , z, sigma, b, r)
        k2 = dy_dt(t + 0.5 * dt, x, y + 0.5 * dt * k1, z, sigma, b, r)
        k3 = dy_dt(t + 0.5 * dt, x, y + 0.5 * dt * k2, z, sigma, b, r)
        k4 = dy_dt(t + dt      , x, y + dt * k3      , z, sigma, b, r)
        y_i = dt * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        
        k1 = dz_dt(t           , x, y, z                , sigma, b, r)
        k2 = dz_dt(t + 0.5 * dt, x, y, z + 0.5 * dt * k1, sigma, b, r)
        k3 = dz_dt(t + 0.5 * dt, x, y, z + 0.5 * dt * k2, sigma, b, r)
        k4 = dz_dt(t + dt      , x, y, z + dt * k3      , sigma, b, r)
        z_i = dt * (k1 + 2 * k2 + 2 * k3 + k4) / 6 
        
        t += dt
        x += x_i
        y += y_i
        z += z_i
        
        ts.append(t)
        xs.append(x)
        ys.append(y)
        zs.append(z)
    
    return ts, xs, ys, zs

In [9]:
def adams_y(x, y, z, t, h, step, f, r):
    return h * (
        55 * f(t[step - 1], x[step - 1], y[step - 1], z[step - 1], r) - 
        59 * f(t[step - 2], x[step - 2], y[step - 2], z[step - 2], r) + 
        37 * f(t[step - 3], x[step - 3], y[step - 3], z[step - 3], r) - 
        9 * f(t[step - 4], x[step - 4], y[step - 4], z[step - 4], r)
    ) / 24

def adams_dx_dt(t, x, y, z, r):
    return dx_dt(t, x, y, z, sigma, b, r)

def adams_dy_dt(t, x, y, z, r):
    return dy_dt(t, x, y, z, sigma, b, r)

def adams_dz_dt(t, x, y, z, r):
    return dz_dt(t, x, y, z, sigma, b, r)

def solve_adams(r, t0=0, x0=13.41265629, y0=13.46430003, z0=33.46156416):
    t = t0
    x = x0
    y = y0
    z = z0
    
    ts, xs, ys, zs = tuple(map(lambda x: x[:4], RK4(r)))
    step = 4
    
    while t < max_t:
        new_x = adams_y(xs, ys, zs, ts, dt, step, adams_dx_dt, r)
        new_y = adams_y(xs, ys, zs, ts, dt, step, adams_dy_dt, r)
        new_z = adams_y(xs, ys, zs, ts, dt, step, adams_dz_dt, r)
        
        t += dt
        x += new_x
        y += new_y
        z += new_z
        
        ts.append(t)
        xs.append(x)
        ys.append(y)
        zs.append(z)
        
        step += 1
    
    return ts, xs, ys, zs

In [ ]:
draw(solve_adams)

In [13]:
def adams_multan_y(x, y, z, t, h, step, f, r):
    return h * (
        9 * f(t[step], x[step], y[step], z[step], r) +
        19 * f(t[step - 1], x[step - 1], y[step - 1], z[step - 1], r) - 
        5 * f(t[step - 2], x[step - 2], y[step - 2], z[step - 2], r) + 
        f(t[step - 3], x[step - 3], y[step - 3], z[step - 3], r)
    ) / 24

def solve_adams_non_evident(r, t0=0, x0=13.41265629, y0=13.46430003, z0=33.46156416):
    t = t0
    x = x0
    y = y0
    z = z0
    
    ts, xs, ys, zs = tuple(map(lambda x: x[:4], RK4(r)))
    step = 4
    
    while t < max_t:
        new_x = xs + [adams_y(xs, ys, zs, ts, dt, step, adams_dx_dt, r)]
        new_y = ys + [adams_y(xs, ys, zs, ts, dt, step, adams_dy_dt, r)]
        new_z = zs + [adams_y(xs, ys, zs, ts, dt, step, adams_dz_dt, r)]
        new_t = ts + [t + dt]
        t += dt
        x += adams_multan_y(new_x, new_y, new_z, new_t, dt, step, adams_dx_dt, r)
        y += adams_multan_y(new_x, new_y, new_z, new_t, dt, step, adams_dy_dt, r)
        z += adams_multan_y(new_x, new_y, new_z, new_t, dt, step, adams_dz_dt, r)
        
        ts.append(t)
        xs.append(x)
        ys.append(y)
        zs.append(z)
        
        step += 1
    
    return ts, xs, ys, zs

In [ ]:
draw(solve_adams_non_evident)